In [2]:
import numpy as np
import pandas as pd
import nltk
import gensim
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
from gensim import corpora, models
import re
stemmed_tokens=[]
import string
from nltk.corpus import stopwords

In [3]:
articles_data=pd.read_csv("news_articles.csv")
article=list(articles_data['Content'])

In [4]:
for i in range(4831):
    raw=article[i].lower()
    raw=re.sub(r'\d+', '', raw)        #removing punctutation
    word_tokens = tokenizer.tokenize(raw)   #tokens
    stpwrd_sentence = [w for w in word_tokens if not w in stop_words] #remove stopwords is,am,are..... common words
    lemmatised_words=[lemmatizer.lemmatize(w) for w in stpwrd_sentence]  #lemmitizer to get root form of word
    stemmed_words=[stemmer.stem(i) for i in lemmatised_words]            #stemming
    stemmed_tokens.append(stemmed_words)   


In [5]:
dictionary=corpora.Dictionary(stemmed_tokens)
corpus=[dictionary.doc2bow(d) for d in stemmed_tokens]             #count how many times word occur
#corpus                     #lists of document 1 to n 

In [6]:
ldamodel=gensim.models.ldamodel.LdaModel(corpus,num_topics=4,id2word=dictionary,passes=20,minimum_probability=0.0)

In [7]:
topics=[]
for i in range(4831):
    item=ldamodel[corpus[i]]                      #gives probabiloty of i document in topics 0,1,2,3
    item=np.array(item).T                       #for computation convert list to array and transpose to get 2*4
    item=item[1]                                  #get probabilites  1*4
    topics.append(item)                               # append  probabilites 
topics_array=np.array(topics)

In [8]:
#kmeans 
import sklearn
from sklearn.cluster import KMeans
kmeans=sklearn.cluster.KMeans(4).fit(topics)    #clusters using topics(probabilty of matric  n*4  (n=documents,4=topics))
clusters=kmeans.labels_

In [162]:
#add length column
length=[]
for i in range(4831):
    length.append(len(articles_data["Content"][i]))
    
articles_data["Length"]=length
#add average time
ti=[]
for i in range(4831):
    ti.append(int(length[i]/5))
articles_data["avg_Time"]=ti

In [163]:
noofusers=int(input("enter users: "))

enter users: 10


# 

In [164]:
#add user ids
pls=[]
for i in range(1,noofusers+1):
    for j in range(8):
        p=1*i
        pls.append(p)
    

In [165]:
datathrough=pd.DataFrame()                                                   #articles alloted to users
for i in range (noofusers):
    for i in range(4):
        user=(articles_data[clusters==i][:2])
        datathrough=datathrough.append(user)

In [166]:
datathrough.to_pickle("articlethrough.pkl")
#pd.read_pickle("articlethrough.pkl") read from pickle  
#datathrough

In [167]:
#genrative

In [168]:
 def timefxn(user_profile):
    time1=np.random.normal(20,0.3,2000)
    time2=np.random.normal(30,0.3,2000)
    time3=np.random.normal(100,0.3,831)
    total_time=np.append(time1,time2)
    time=np.append(total_time,time3)
    actual=np.random.choice(time,len(user_profile["Article_Id"]))
    return actual

In [169]:
def checkfxn(user_profile,hey,actual,noofusers):
    check=[]            #yes or no and add time
    actual_time=[]
    for j in range(1,noofusers+1):
        for i in range(8):
            if list(user_profile["Article_Id"][user_profile["User_id"]==j])[i] in list(hey['Article_Id'][hey["User_id"]==j]):
                check.append("YES")
                actual_time.append(int(actual[i*j]))
            
            else:
                check.append("NO")
                actual_time.append(0)
    return check,actual_time

In [170]:
def addweight(user_info):
    import math             #finding weight
    user_info["ratio"]=user_info["actual_time"]/user_info["average_Time"]
    weight=[]
    for i in user_info["ratio"]:
        if i==0:
            weight.append(0)
        else:
            weight.append(1/(1+math.exp(-i)))
    user_info["weight"]=weight
    user_info.drop(["ratio"],axis=1,inplace=True)
    probability=[]
    for i in range(len(user_info["articleID"])):
        yo=weight[i]*topics[user_info["articleID"][i]]
        probability.append(yo)
    user_info["weight*prob"]=probability
    
    return user_info

In [171]:
def ourfxn(noofusers,datathrough):                                          #fxn on through artciles
    hey=pd.DataFrame()
    datathrough["User_id"]=pls
    user_prof=datathrough
    user_profile=user_prof[["Article_Id",'Title','Length',"User_id","avg_Time"]]
    number=[]
    for i in range(noofusers):
        no_of_articles=np.random.poisson(3,1)[0]
        number.append(no_of_articles)
    time=timefxn(user_profile)
    

    for i in range(1,noofusers+1):
        hey=hey.append(user_profile[user_profile["User_id"]==i].sample(number[i-1],replace=True)) #throw n samples from userr(dataframe of 8 articles ,2 from each topics)
    #print(user_profile)
    check,actual_time=checkfxn(user_profile,hey,time,noofusers)
    user_info=pd.DataFrame({"articleID":list(user_profile["Article_Id"]),"Title":list(user_profile["Title"]),"Length":list(user_profile["Length"]),"User_id":list(user_profile["User_id"]),"check":check,"average_Time":list(user_profile["avg_Time"]),"actual_time":actual_time})
    user_info=addweight(user_info)
    return user_info

In [172]:
#new_info=ourfxn(2)
#new_info.to_pickle("userinfo.pkl")

In [187]:
from sklearn.metrics.pairwise import cosine_similarity         #articles suggested after cosimlarity
def cosim(noofusers,user_info,session):
    b=[]
    #user_info=pd.read_pickle("userinfo.pkl")
    userspro=[]                                 #profile of users after summing the prob
    if session == 2:
        for i in range(1,noofusers+1):
            userspro.append(np.sum(user_info["weight*prob"][user_info["User_id"]==i]))
        with open("parrots.pkl",'wb') as fu:       #save user profile
            pickle.dump(userspro,fu)
    else:
        users=[]
        with open("parrots.pkl",'rb') as fu:        #read last session user profile
            lists=pickle.load(fu)
        for i in range(1,noofusers+1):
            users.append(np.sum(user_info["weight*prob"][user_info["User_id"]==i]))
        userspro=(np.array(lists)+np.array(users))/2
        
        
        
    suggestarticles=[]
    for i in userspro:
        suggestarticles.append(np.argsort(cosine_similarity(topics,i.reshape(1,-1)).flatten())[-8:][::-1])      #suggest 8 articles
    for i in range(noofusers):
        for j in range(8):
            b.append(suggestarticles[i][j])                      #list of articles suggested
            
    return articles_data.iloc[b]  

In [191]:
session=int(input("enter session: "))

enter session: 3


In [193]:
for i in range(1,session+1):
    if i ==1:
        data=ourfxn(noofusers,datathrough)
        data["session"]=np.ones(8*noofusers)*i
        data.to_pickle("newdata.pkl")
    if i ==2 :
        data1=pd.read_pickle("newdata.pkl")
        data2=cosim(noofusers,data1,i)
        data3=ourfxn(noofusers,data2)
        data3["session"]=np.ones(8*noofusers)*i
        data=data1.append(data3)
        data.to_pickle("newdata.pkl")
    if i >=3:
        data1=pd.read_pickle("newdata.pkl")
        data2=cosim(noofusers,data1,i)
        data3=ourfxn(noofusers,data2)
        data3["session"]=np.ones(8*noofusers)*i
        data=data1.append(data3)
        data.to_pickle("newdata.pkl")
        
        
        
    
        

G:\Newfolder\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\Newfolder\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [194]:
pd.read_pickle("newdata.pkl")

,articleID,Title,Length,User_id,check,average_Time,actual_time,weight,weight*prob,session
0,6,Infosys shares likely to fall on Tuesday after...,2104,1,NO,420,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
1,30,Box office collection Jil Jung Juk Deadpo...,2144,1,NO,428,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
2,0,14 dead after bus falls into canal in Telangan...,1355,1,YES,271,30,0.527647,"[0.12260186953100466, 0.04461321879561946, 0.3...",1.0
3,1,Pratibha Tiwari molested on busy road Saath ...,1215,1,NO,243,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
4,238,Su Su Sudhi Vathmeekam review Live audience...,55,1,NO,11,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
5,249,Premam ZooZoo Version Watch Funny Ad Creatu...,2026,1,NO,405,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
6,2,US South Korea begin joint military drill ami...,1369,1,NO,273,0,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.0
7,9,Rajasthan Villagers rescue 50 kids after scho...,661,1,YES,132,100,0.680827,"[0.2996642898111763, 0.08595683420716776, 0.29...",1.0
8,6,Infosys shares likely to fall on Tuesday after...,2104,2,YES,420,20,0.511903,"[0.012767790201070687, 0.48756742555329535, 0....",1.0
9,30,Box office collection Jil Jung Juk Deadpo...,2144,2,YES,428,30,0.517516,"[0.000624432684728873, 0.27558895524189236, 0....",1.0
